In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys
import pymedphys._mocks.profiles
import pymedphys._gamma.implementation.shell
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findbb


In [ ]:
image_path = pymedphys.data_path('wlutz_image.png')
image_path

In [ ]:
# img = imageio.imread(image_path)
# assert np.shape(img) == (1024, 1024)
# img = img[:, 1:-1]
# assert np.shape(img) == (1024, 1022)
# assert img.dtype == np.dtype('uint16')
# img = 1 - img[::-1,:] / 2**16
# assert img.dtype == np.dtype('float64')

In [ ]:
x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)
field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)

In [ ]:
edge_lengths = [20, 20]

centre, rotation = pymedphys._wlutz.findfield.find_centre_and_rotation(x, y, img, edge_lengths)
centre, rotation

In [ ]:
def draw_by_diff(dx, dy, transform):
    draw_x = np.cumsum(dx)
    draw_y = np.cumsum(dy)

    draw_x, draw_y = pymedphys._wlutz.interppoints.apply_transform(draw_x, draw_y, transform)
    
    return draw_x, draw_y

In [ ]:
transform = matplotlib.transforms.Affine2D()
transform.rotate_deg(-rotation)
transform.translate(*centre)

In [ ]:
rotation_x_points = np.linspace(-edge_lengths[0]/2, edge_lengths[0]/2, 51)
rotation_y_points = np.linspace(-edge_lengths[1]/2, edge_lengths[1]/2, 61)

rot_xx_points, rot_yy_points = np.meshgrid(rotation_x_points, rotation_y_points)

rot_xx_points, rot_yy_points = pymedphys._wlutz.interppoints.apply_transform(rot_xx_points, rot_yy_points, transform)

In [ ]:

rotation_points_at_origin = pymedphys._wlutz.interppoints.define_rotation_field_points_at_origin([20, 20], 2)
rotation_points = pymedphys._wlutz.interppoints.transform_rotation_field_points(rotation_points_at_origin, centre, rotation)

In [ ]:
rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

plt.figure(figsize=(10,10))
plt.pcolormesh(x, y, img)
plt.plot(*draw_by_diff(rect_dx, rect_dy, transform), 'k', lw=2)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, transform), 'k', lw=0.5)

plt.scatter(centre[0], centre[1], c='r', s=1)

plt.scatter(rotation_points[0], rotation_points[1], s=1)

plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])

In [ ]:
# def create_points_to_check(ball_bearing_diameter):
#     min_dist = 0.5
#     distances = np.arange(0, ball_bearing_diameter, min_dist)

#     x = []
#     y = []
#     dist = []
    
#     for _, distance in enumerate(distances):
#         new_x, new_y = pymedphys._gamma.implementation.shell.calculate_coordinates_shell_2d(distance, min_dist)
#         x.append(new_x)
#         y.append(new_y)
#         dist.append(distance * np.ones_like(new_x))

#     x = np.concatenate(x)
#     y = np.concatenate(y)
#     dist = np.concatenate(dist)
    
#     def points_to_check(bb_centre):
#         x_shifted = x + bb_centre[0]
#         y_shifted = y + bb_centre[1]
        
#         return x_shifted, y_shifted
    
#     return points_to_check, dist


# points_to_check, dist = create_points_to_check(8)
# x, y = points_to_check([1.46, -1.9])
# plt.scatter(x, y, s=3)
# plt.axis('equal')

In [ ]:
# dist_mask = (np.unique(dist)[:, None] == dist[None, :])

# x[dist_mask[1]]

In [ ]:
# total_minimisation = 0

# previous_mask = dist_mask[0]
# previous_field = field(x[previous_mask], y[previous_mask])
# mean_previous_field = np.mean(previous_field)

# for current_mask in dist_mask[1::]:
#     current_field = field(x[current_mask], y[current_mask])
#     mean_current_field = np.mean(current_field)

#     between_layer_diff = np.mean((mean_previous_field - current_field) ** 3)
#     current_layer_sim = np.mean((current_field - mean_current_field)**2)
    
#     total_minimisation += between_layer_diff + current_layer_sim
    
#     mean_previous_field = mean_current_field

    
# total_minimisation

In [ ]:
# dist[None, :] == dist[:, None]

In [ ]:
# def create_bb_to_minimise(field, ball_bearing_diameter):
    
#     points_to_check, dist = create_points_to_check(ball_bearing_diameter)
#     dist_mask = np.unique(dist)[:, None] == dist[None, :]
    
#     def to_minimise(centre):
#         x, y = points_to_check(centre)
        
#         total_minimisation = 0

#         previous_mask = dist_mask[0]
#         previous_field = field(x[previous_mask], y[previous_mask])
#         mean_previous_field = np.mean(previous_field)

#         for current_mask in dist_mask[1::]:
#             current_field = field(x[current_mask], y[current_mask])
#             mean_current_field = np.mean(current_field)

#             between_layer_diff = np.mean(((mean_previous_field - current_field) / mean_previous_field) ** 3)
#             current_layer_sim = np.mean(((current_field - mean_current_field) / mean_current_field)**2)

#             total_minimisation += between_layer_diff + current_layer_sim

#             mean_previous_field = mean_current_field
        
#         return total_minimisation
    
#     return to_minimise

In [ ]:
ball_bearing_diameter = 8
to_minimise = pymedphys._wlutz.findbb.create_bb_to_minimise(field, ball_bearing_diameter)

In [ ]:
t = np.linspace(0, 2*np.pi)
circle_x_origin = ball_bearing_diameter/2 * np.sin(t)
circle_y_origin = ball_bearing_diameter/2 * np.cos(t)

In [ ]:
def rect_points(edge_lengths):
    rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
    rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]
    
    draw_x = np.cumsum(dx)
    draw_y = np.cumsum(dy)
    
    return draw_x, draw_y

In [ ]:
bb_centre = [2, -1]

circle_x = circle_x_origin + bb_centre[0]
circle_y = circle_y_origin + bb_centre[1]


rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

plt.figure(figsize=(10,10))
plt.pcolormesh(x, y, img)
plt.plot(*draw_by_diff(rect_dx, rect_dy, transform), 'k', lw=2)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, transform), 'k', lw=0.5)

plt.scatter(centre[0], centre[1], c='r', s=1)

plt.scatter(rotation_points[0], rotation_points[1], s=1)

plt.scatter(*bb_centre)
plt.plot(circle_x, circle_y, 'k', lw=2)

plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])


to_minimise(bb_centre)

In [ ]:
to_minimise

In [ ]:
def create_centralised_field(field, centre, rotation):
    
    transform = pymedphys._wlutz.interppoints.translate_and_rotate_transform(centre, rotation)
    
    def new_field(x, y):
        x_prime, y_prime = pymedphys._wlutz.interppoints.apply_transform(x, y, transform)
        return field(x_prime, y_prime)
    
    return new_field


In [ ]:
centralised_field = create_centralised_field(field, centre, rotation)

In [ ]:
xx, yy = np.meshgrid(x, y)

plt.pcolormesh(x, y, centralised_field(xx, yy))
plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])

In [ ]:
penumbra = 2
bb_diameter =8

In [ ]:
half_field_bounds = [
    (edge_lengths[0] - penumbra / 2) / 2, (edge_lengths[1] - penumbra / 2) / 2]

bb_radius = bb_diameter / 2

circle_centre_bounds = [
    (
        -half_field_bounds[0] + bb_radius,
        half_field_bounds[0] - bb_radius
    ),
    (
        -half_field_bounds[1] + bb_radius,
        half_field_bounds[1] - bb_radius
    ),
]



In [ ]:
ball_bearing_diameter = 8
centralised_to_minimise = pymedphys._wlutz.findbb.create_bb_to_minimise(centralised_field, ball_bearing_diameter)
to_minimise = pymedphys._wlutz.findbb.create_bb_to_minimise(field, ball_bearing_diameter)

In [ ]:
bb_results = scipy.optimize.basinhopping(
    centralised_to_minimise, [0, 0], T=1, niter=200, niter_success=5, stepsize=0.25,
    minimizer_kwargs={
        'method': 'L-BFGS-B',
        'bounds': circle_centre_bounds
    }
)
bb_centre_in_centralised_field = bb_results.x

bb_centre_in_centralised_field

In [ ]:
transform = pymedphys._wlutz.interppoints.translate_and_rotate_transform(centre, rotation)
bb_centre = pymedphys._wlutz.interppoints.apply_transform(*bb_centre_in_centralised_field, transform)
bb_centre

In [ ]:
to_minimise(bb_centre)

In [ ]:
to_minimise([1.49, 2])

In [ ]:
x_half_bound = edge_lengths[0]/2 + penumbra * 3
y_half_bound = edge_lengths[1]/2 + penumbra * 3


x_axis = np.linspace(-x_half_bound, x_half_bound, 200)
y_axis = np.linspace(-y_half_bound, y_half_bound, 200)

x_field_interp, y_field_interp = pymedphys._wlutz.createaxis.transform_axis(x_axis, y_axis, centre, rotation)
x_bb_interp, y_bb_interp = pymedphys._wlutz.createaxis.transform_axis(x_axis, y_axis, bb_centre, 0)

# plt.plot(x_interp[0], x_interp[1], label='x-axis')
# plt.plot(y_interp[0], y_interp[1], label='y-axis')
# # plt.pcolormesh(xx, yy, zz)

# plt.legend()
# plt.axis('equal')

In [ ]:
# bb_centre = [1.5, 1]

bb_radius = bb_diameter / 2

circle_x = circle_x_origin + bb_centre[0]
circle_y = circle_y_origin + bb_centre[1]


rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

bb_crosshair = np.array([-bb_radius, bb_radius])

plt.figure(figsize=(10,10))
plt.contourf(x, y, img, 100)

plt.plot([bb_centre[0], ]*2, bb_crosshair + bb_centre[1], 'k', lw=1)
plt.plot(bb_crosshair + bb_centre[0], [bb_centre[1], ]*2, 'k', lw=1)

plt.plot(*draw_by_diff(rect_dx, rect_dy, transform), 'k', lw=3)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, transform), 'k', lw=1)

plt.plot([centre[0], bb_centre[0]], [centre[1], bb_centre[1]], c='C3', lw=3)

# plt.scatter(rotation_points[0], rotation_points[1], s=1)

# plt.scatter(*bb_centre)
plt.plot(circle_x, circle_y, 'k', lw=3)

plt.plot(x_field_interp[0], x_field_interp[1], 'k', lw=0.5, alpha=0.3)
plt.plot(y_field_interp[0], y_field_interp[1], 'k', lw=0.5, alpha=0.3)

plt.plot(x_bb_interp[0], x_bb_interp[1], 'k', lw=0.5, alpha=0.3)
plt.plot(y_bb_interp[0], y_bb_interp[1], 'k', lw=0.5, alpha=0.3)

plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])

plt.colorbar()

to_minimise(bb_centre)



In [ ]:
plt.plot(x_axis, field(*x_field_interp))
plt.plot(-x_axis, field(*x_field_interp))

In [ ]:
plt.plot(y_axis, field(*y_field_interp))
plt.plot(-y_axis, field(*y_field_interp))

In [ ]:
plt.plot(x_axis, field(*x_bb_interp))
plt.plot(-x_axis, field(*x_bb_interp))

In [ ]:
plt.plot(y_axis, field(*y_bb_interp))
plt.plot(-y_axis, field(*y_bb_interp))
plt.plot([-bb_diameter/2, bb_diameter/2], [0.6, 0.6])

In [ ]:
x_bb_interp

In [ ]:
np.array(bb_centre) - np.array(centre)

In [ ]:
bb_centre

In [ ]:
plt.pcolormesh(x, y, centralised_field(xx, yy))


plt.plot(circle_centre_bounds[0], circle_centre_bounds[1], '.')
plt.plot(circle_centre_bounds[0][::-1], circle_centre_bounds[1], '.')

plt.plot(half_field_bounds[0], half_field_bounds[1], '.')

plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])